In [1]:
# drive.flush_and_unmount()
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/cs498/assignment5_materials")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Deep Q-Learning 

#### Install dependencies for AI gym to run properly (shouldn't take more than a minute). If running on google cloud or running locally, only need to run once. Colab may require installing everytime the vm shuts down.

In [2]:
!pip3 install gym pyvirtualdisplay
!sudo apt-get install -y xvfb python-opengl ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [3]:
!pip3 install --upgrade setuptools
!pip3 install ez_setup 
!pip3 install 'gym[atari]'
# !pip3 install torch

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (51.0.0)


For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [4]:
%matplotlib inline

import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import find_max_lives, check_live, get_frame, get_init_state
from model import DQN
from config import *

import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialize our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In breakout, we will use 3 actions "fire", "left", and "right". "fire" is only used to reset the game when a life is lost, "left" moves the agent left and "right" moves the agent right.

In [5]:
env = gym.make('BreakoutDeterministic-v4')
state = env.reset()

In [6]:
number_lives = find_max_lives(env)
state_size = env.observation_space.shape
action_size = 3 #fire, left, and right

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. Once you've created a working DQN agent, use the code in agent.py to create a double DQN agent in __agent_double.py__. Set the flag "double_dqn" to True to train the double DQN agent.

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [7]:
double_dqn = True # set to True if using double DQN agent

if double_dqn:
    from agent_double import Agent
else:
    from agent import Agent

agent_007 = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

### Main Training Loop

In this training loop, we do not render the screen because it slows down training signficantly. To watch the agent play the game, run the code in next section "Visualize Agent Performance"

In [ ]:
rewards, episodes = [], []
best_eval_reward = 0
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    next_state = state
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1

        # Perform a fire action if ball is no longer on screen to continue onto next life
        if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
            action = 0
        else:
            action = agent_007.get_action(np.float32(history[:4, :, :]) / 255.)
        state = next_state
        next_state, reward, done, info = env.step(action + 1)
        
        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = np.clip(reward, -1, 1) 
        r = reward

        # Store the transition in memory 
        agent_007.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
#         if(frame >= 100):
            agent_007.train_policy_net(frame)
            # Update the target network only for Double DQN only
            if double_dqn and (frame % update_target_network_frequency)== 0:
                agent_007.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]
            
        if done:
            evaluation_reward.append(score)
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.xlabel('Episodes')
            pylab.ylabel('Rewards') 
            pylab.title('Episodes vs Reward')
            if double_dqn:
                pylab.savefig("./save_graph/breakout_ddqn.png") # save graph for training visualization
            else:
                pylab.savefig("./save_graph/breakout_dqn.png")
            
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent_007.memory), "  epsilon:", agent_007.epsilon, "   steps:", step,
                  "   lr:", agent_007.optimizer.param_groups[0]['lr'], "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 100 episode is bigger than 5 save model
            ### Change this save condition to whatever you prefer ###
            if np.mean(evaluation_reward) > 5 and np.mean(evaluation_reward) > best_eval_reward:
                if double_dqn:
                    torch.save(agent_007.policy_net, "./save_model/breakout_ddqn.pth")
                else:
                    torch.save(agent_007.policy_net, "./save_model/breakout_dqn.pth")
                best_eval_reward = np.mean(evaluation_reward)


episode: 0   score: 2.0   memory length: 219   epsilon: 1.0    steps: 219    lr: 0.0001     evaluation reward: 2.0
episode: 1   score: 3.0   memory length: 469   epsilon: 1.0    steps: 250    lr: 0.0001     evaluation reward: 2.5
episode: 2   score: 1.0   memory length: 639   epsilon: 1.0    steps: 170    lr: 0.0001     evaluation reward: 2.0
episode: 3   score: 4.0   memory length: 898   epsilon: 1.0    steps: 259    lr: 0.0001     evaluation reward: 2.5
episode: 4   score: 1.0   memory length: 1067   epsilon: 1.0    steps: 169    lr: 0.0001     evaluation reward: 2.2
episode: 5   score: 2.0   memory length: 1266   epsilon: 1.0    steps: 199    lr: 0.0001     evaluation reward: 2.1666666666666665
episode: 6   score: 2.0   memory length: 1484   epsilon: 1.0    steps: 218    lr: 0.0001     evaluation reward: 2.142857142857143
episode: 7   score: 0.0   memory length: 1606   epsilon: 1.0    steps: 122    lr: 0.0001     evaluation reward: 1.875
episode: 8   score: 3.0   memory length: 1873

/content/gdrive/My Drive/cs498/assignment5_materials/agent_double.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  next_state_values[mask] = self.target_net(next_states[mask]).max(1)[0].detach()


episode: 543   score: 4.0   memory length: 100250   epsilon: 0.9995030200000108    steps: 299    lr: 0.0001     evaluation reward: 1.47
episode: 544   score: 4.0   memory length: 100512   epsilon: 0.998984260000022    steps: 262    lr: 0.0001     evaluation reward: 1.5
episode: 545   score: 0.0   memory length: 100635   epsilon: 0.9987407200000273    steps: 123    lr: 0.0001     evaluation reward: 1.5
episode: 546   score: 5.0   memory length: 100973   epsilon: 0.9980714800000419    steps: 338    lr: 0.0001     evaluation reward: 1.51
episode: 547   score: 1.0   memory length: 101123   epsilon: 0.9977744800000483    steps: 150    lr: 0.0001     evaluation reward: 1.48
episode: 548   score: 1.0   memory length: 101292   epsilon: 0.9974398600000556    steps: 169    lr: 0.0001     evaluation reward: 1.49
episode: 549   score: 1.0   memory length: 101461   epsilon: 0.9971052400000628    steps: 169    lr: 0.0001     evaluation reward: 1.48
episode: 550   score: 1.0   memory length: 101630  

# Visualize Agent Performance

BE AWARE THIS CODE BELOW MAY CRASH THE KERNEL IF YOU RUN THE SAME CELL TWICE.

Please save your model before running this portion of the code.

In [ ]:
# torch.save(agent_007.policy_net, "./save_model/breakout_dqn_latest.pth")
agent_007.load_policy_net("./save_model/breakout_dqn.pth")

In [ ]:
from gym.wrappers import Monitor
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display

# Displaying the game live
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % ("Agent Playing",step, info))
    plt.axis('off')

    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())
    
# Recording the game and replaying the game afterwards
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

In [ ]:
display = Display(visible=0, size=(300, 200))
display.start()

# Load agent
# agent_007.load_policy_net("./save_model/breakout_dqn.pth")
agent_007.epsilon = 0.0 # Set agent to only exploit the best action

env = gym.make('BreakoutDeterministic-v4')
env = wrap_env(env)

done = False
score = 0
step = 0
state = env.reset()
next_state = state
life = number_lives
history = np.zeros([5, 84, 84], dtype=np.uint8)
get_init_state(history, state)

while not done:
    
    # Render breakout
    env.render()
#     show_state(env,step) # uncommenting this provides another way to visualize the game

    step += 1
    frame += 1

    # Perform a fire action if ball is no longer on screen
    if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
        action = 0
    else:
        action = agent_007.get_action(np.float32(history[:4, :, :]) / 255.)
    state = next_state
    
    next_state, reward, done, info = env.step(action + 1)
        
    frame_next_state = get_frame(next_state)
    history[4, :, :] = frame_next_state
    terminal_state = check_live(life, info['ale.lives'])
        
    life = info['ale.lives']
    r = np.clip(reward, -1, 1) 
    r = reward

    # Store the transition in memory 
    agent_007.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
    # Start training after random sample generation
    score += reward
    
    history[:4, :, :] = history[1:, :, :]
env.close()
show_video()
display.stop()